<a href="https://colab.research.google.com/github/jcmachicao/intro_ia_maestrias_espa-a/blob/main/Python_vs_IA_Colaborativo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python vs IA: Exploración Colaborativa

Este notebook acompaña la comparación entre el proceso con Python puro y el proceso asistido por IA. Incluye espacio para análisis, experimentación y reflexiones guiadas.

In [31]:
import pandas as pd
import numpy as np

np.random.seed(42)

dates = pd.date_range("2024-01-01", periods=200)
categories = ["A", "B", "C"]

data = {
    "fecha": np.random.choice(dates, 200),
    "categoria": np.random.choice(categories, 200),
    "precio": np.random.uniform(10, 200, 200),
    "cantidad": np.random.randint(1, 20, 200)
}

df = pd.DataFrame(data)
df["total"] = df["precio"] * df["cantidad"]
df.head()


,fecha,categoria,precio,cantidad,total
0,2024-04-12,A,15.247187,15,228.707800
1,2024-06-28,A,119.984330,1,119.984330
2,2024-04-02,B,93.310083,3,279.930250
3,2024-01-15,A,137.684966,16,2202.959451
4,2024-04-16,B,72.349007,11,795.839075


In [38]:
df.groupby("categoria")["total"].sum()

,total
categoria,
A,73583.826587
B,74486.457971
C,68273.027942


In [40]:
suma_categorias = df.groupby("categoria")["total"].sum()
categoria_mas_rentable = suma_categorias.idxmax()
print(f"La categoría más rentable es: {categoria_mas_rentable}")

La categoría más rentable es: B


In [41]:
from getpass import getpass
from openai import OpenAI
from google.colab import userdata

#api_key = getpass("Introduce tu OPENAI_API_KEY: ")
api_key = userdata.get('OAIK_JCMV')
client = OpenAI(api_key=api_key)

In [42]:
prompt = """
Eres un asistente experto en análisis con pandas.
Tu trabajo es generar SOLO código Python válido.
No expliques nada.
No describas los datos porque no los ves.
El DataFrame se llama df.

Cuando recibas una pregunta sobre el análisis,
responde únicamente con código ejecutable.
"""

In [43]:
def consulta_llm(pregunta):
    message = prompt + "\nPregunta: " + pregunta

    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[{"role": "user", "content": message}]
    )

    generated = response.choices[0].message.content
    print("Código sugerido por el modelo:\n")
    print(generated)
    return generated

In [45]:
columnas = list(df.columns)
pregunta = f"Dadas las columnas del df, que son {columnas}, ¿Cuál es la categoría más rentable?"
codigo = consulta_llm(pregunta)

Código sugerido por el modelo:

```python
df.groupby('categoria')['total'].sum().idxmax()
```


In [46]:
cleaned_codigo = "df.groupby('categoria')['total'].sum().idxmax()"
print(eval(cleaned_codigo))

B


# La enorme utilidad del formato JSON

In [47]:
import json
import re

messy_text = """\n    En la tienda, vi que tenían una Laptop Pro disponible. ¡Imagínate, un precio unitario de 1200.50! Y según el encargado, aún les quedaban unas 50 unidades en stock. El proveedor principal, ElectroCorp, con su famoso Id ELC001, la distribuye. Ah, y me dijeron que es el modelo 2023 con una pantalla de 15.6 pulgadas, bastante impresionante.\n\n    Más tarde, mientras buscaba accesorios, encontré un Teclado Mecánico. El costo era de solo 75.00, y me sorprendió que tuvieran como 200 unidades disponibles. El suministrador de estos era TechGears (ID_prov: TCHG005). Me gustó que tuviera iluminación RGB y switches rojos, además de ser compatible con Windows y Mac, ¡muy versátil! Parece que tienen un buen surtido.\n    """

def parse_product_info(text):
    products = []
    # Split by double newline to get individual product narratives (assuming each product is in a paragraph)
    narrative_entries = [entry.strip() for entry in text.strip().split('\n\n') if entry.strip()]

    for entry_text in narrative_entries:
        product_data = parse_single_narrative_entry(entry_text)
        if product_data: # Only add if data was successfully extracted
            products.append(product_data)
    return products

def parse_single_narrative_entry(entry_text):
    product_data = {}

    # Extract product name (more flexible patterns)
    name_match_laptop = re.search(r'(?:una|un)\s+(Laptop Pro)', entry_text, re.IGNORECASE)
    name_match_keyboard = re.search(r'(?:un|un)\s+(Teclado Mecánico)', entry_text, re.IGNORECASE)

    if name_match_laptop:
        product_data['nombre'] = name_match_laptop.group(1).strip()
    elif name_match_keyboard:
        product_data['nombre'] = name_match_keyboard.group(1).strip()

    # Extract price
    price_match = re.search(r'(?:precio unitario de|costo era de solo)\s+([\d.]+)', entry_text)
    if price_match:
        product_data['precio'] = float(price_match.group(1))

    # Extract stock
    stock_match = re.search(r'(?:unas|como)\s+([\d]+)\s+unidades\s+(?:en stock|disponibles)', entry_text)
    if stock_match:
        product_data['stock'] = int(stock_match.group(1))

    # Extract supplier
    # This pattern tries to capture the supplier name before a comma, parenthesis, or period
    supplier_match = re.search(r'(?:El proveedor principal,|El suministrador de estos era)\s+([\w\s.-]+?)(?:, con su famoso Id|\(ID_prov:|\.)', entry_text)
    if supplier_match:
        product_data['proveedor'] = supplier_match.group(1).strip()

    return product_data


structured_data = parse_product_info(messy_text)

# Convert to JSON
json_output = json.dumps(structured_data, indent=4, ensure_ascii=False)

print("Texto Desestructurado Original:")
print(messy_text)
print("\n--- Convertido a JSON ---")
print(json_output)


Texto Desestructurado Original:

    En la tienda, vi que tenían una Laptop Pro disponible. ¡Imagínate, un precio unitario de 1200.50! Y según el encargado, aún les quedaban unas 50 unidades en stock. El proveedor principal, ElectroCorp, con su famoso Id ELC001, la distribuye. Ah, y me dijeron que es el modelo 2023 con una pantalla de 15.6 pulgadas, bastante impresionante.

    Más tarde, mientras buscaba accesorios, encontré un Teclado Mecánico. El costo era de solo 75.00, y me sorprendió que tuvieran como 200 unidades disponibles. El suministrador de estos era TechGears (ID_prov: TCHG005). Me gustó que tuviera iluminación RGB y switches rojos, además de ser compatible con Windows y Mac, ¡muy versátil! Parece que tienen un buen surtido.
    

--- Convertido a JSON ---
[
    {
        "nombre": "Laptop Pro",
        "precio": 1200.5,
        "stock": 50,
        "proveedor": "ElectroCorp"
    },
    {
        "nombre": "Teclado Mecánico",
        "precio": 75.0,
        "stock": 200,
 

In [48]:
llm_system_prompt = """
Eres un asistente experto en extracción de información. Tu única tarea es extraer detalles de productos de un texto dado y formatearlos como un arreglo JSON. SOLO debes responder con el arreglo JSON, sin texto adicional, explicaciones, o bloques de código.

Cada objeto en el arreglo JSON debe tener las siguientes claves: 'nombre' (string), 'precio' (float), 'stock' (integer), y 'proveedor' (string). Si alguna información no se encuentra, omite esa clave para ese producto.
"""

llm_user_message = f"""
Extrae los detalles de los productos del siguiente texto y devuélvelos como un arreglo JSON:

{messy_text}
"""

print("Mensaje enviado al LLM:")
print(llm_user_message)

try:
    llm_response = client.chat.completions.create(
        model="gpt-4", # Usando un modelo más capaz para la extracción directa de JSON
        messages=[
            {"role": "system", "content": llm_system_prompt},
            {"role": "user", "content": llm_user_message}
        ]
    )

    # El LLM debería devolver directamente el JSON, pero lo limpiamos por si acaso
    raw_llm_json_output = llm_response.choices[0].message.content.strip()

    # Intentar parsear la cadena JSON
    llm_structured_data = json.loads(raw_llm_json_output)

    print("\n--- JSON generado directamente por el LLM ---")
    print(json.dumps(llm_structured_data, indent=4, ensure_ascii=False))

except json.JSONDecodeError as e:
    print(f"\nError al decodificar JSON del LLM: {e}")
    print("Contenido crudo recibido del LLM:\n", raw_llm_json_output)
except Exception as e:
    print(f"\nOcurrió un error al llamar al LLM o procesar la respuesta: {e}")

Mensaje enviado al LLM:

Extrae los detalles de los productos del siguiente texto y devuélvelos como un arreglo JSON:


    En la tienda, vi que tenían una Laptop Pro disponible. ¡Imagínate, un precio unitario de 1200.50! Y según el encargado, aún les quedaban unas 50 unidades en stock. El proveedor principal, ElectroCorp, con su famoso Id ELC001, la distribuye. Ah, y me dijeron que es el modelo 2023 con una pantalla de 15.6 pulgadas, bastante impresionante.

    Más tarde, mientras buscaba accesorios, encontré un Teclado Mecánico. El costo era de solo 75.00, y me sorprendió que tuvieran como 200 unidades disponibles. El suministrador de estos era TechGears (ID_prov: TCHG005). Me gustó que tuviera iluminación RGB y switches rojos, además de ser compatible con Windows y Mac, ¡muy versátil! Parece que tienen un buen surtido.
    


--- JSON generado directamente por el LLM ---
[
    {
        "nombre": "Laptop Pro",
        "precio": 1200.5,
        "stock": 50,
        "proveedor": "Ele

Para la reflexión:

* El enfoque determinista (NLP/RegEx): Es predecible y preciso cuando las reglas son claras y el formato es relativamente consistente. Los estudiantes ven la lógica manual detrás de la extracción y entienden el esfuerzo de definir cada patrón.

* El enfoque con LLM: Demuestra la inmensa potencia de la IA para procesar lenguaje natural complejo y extraer información de contextos muy variados, incluso con texto muy desordenado. Sin embargo, también abre la discusión sobre la "caja negra" y el potencial de alucinaciones o errores si las instrucciones no son lo suficientemente claras o si el modelo se desvía. Es una excelente forma de introducir el concepto de que una mayor flexibilidad puede conllevar un mayor riesgo (aunque en este caso, el LLM fue muy preciso).

Ambas aproximaciones son valiosas y tienen su lugar, y este ejemplo las contrasta de manera muy efectiva.